# Quantifying Limited Separation of Isolation Forest Scores

This notebook presents **threshold‑independent, overlap‑sensitive** measures insensitive to the sign split:

- **Wasserstein‑1 distance (Earth Mover’s Distance)** on raw scores and on absolute values
- **Cliff’s delta (δ)** as a non‑parametric effect size (computed on `|score|`)
- **Overlap Coefficient (OVL)** between the two densities (histogram/KDE estimate)
- **Proportion within ε of zero** for each class


## 1) Configuration
Choose how to obtain data and set the evaluation periods.

In [1]:
# --- Toggle: set to 'neo4j' to fetch directly, or 'manual' to paste arrays below ---
DATA_SOURCE = 'neo4j'   # 'neo4j' | 'manual'

# --- Neo4j connection (only used if DATA_SOURCE == 'neo4j') ---
NEO4J_URI  = "bolt://192.168.0.5:7687"
NEO4J_USER = "neo4j"
NEO4J_PASS = "PotatoDTND12!"

# --- Score property and label rule ---
SCORE_PROPERTY = 'scoring'      # use 'scoring_v2' for Phase 2
NEGATIVE_IS_ANOMALY = True      # standard Isolation Forest sign-based rule

# --- Periods in America/Sao_Paulo ---
PERIODS = {
    'April 1st 2025': {
        'start': {'year': 2025, 'month': 4, 'day': 1, 'hour': 0, 'minute': 0, 'second': 0},
        'end':   {'year': 2025, 'month': 4, 'day': 2, 'hour': 0, 'minute': 0, 'second': 0},
    },
    'April 2025': {
        'start': {'year': 2025, 'month': 4, 'day': 1, 'hour': 0, 'minute': 0, 'second': 0},
        'end':   {'year': 2025, 'month': 5, 'day': 1, 'hour': 0, 'minute': 0, 'second': 0},
    },
}

# --- Epsilon radii around 0 for proportion-within-epsilon ---
EPSILONS = [1e-6, 1e-5, 1e-4]


## 2) Data access
Fetch scores from Neo4j (America/Sao_Paulo time window) or paste arrays.

In [2]:
import math
import numpy as np

def cypher_for_period(score_prop: str):
    return f'''
WITH datetime({{year:$sy,month:$sm,day:$sd,hour:$sh,minute:$smin,second:$ss, timezone:'America/Sao_Paulo'}}) AS startBR,
     datetime({{year:$ey,month:$em,day:$ed,hour:$eh,minute:$emin,second:$es, timezone:'America/Sao_Paulo'}}) AS endBR
MATCH (tx:Transaction)
WHERE tx.{score_prop} IS NOT NULL
  AND datetime({{epochSeconds: tx.timestamp, timezone:'America/Sao_Paulo'}}) >= startBR
  AND datetime({{epochSeconds: tx.timestamp, timezone:'America/Sao_Paulo'}}) <  endBR
RETURN
  collect(CASE WHEN tx.{score_prop} < 0 THEN tx.{score_prop} END) AS s_anom,
  collect(CASE WHEN tx.{score_prop} > 0 THEN tx.{score_prop} END) AS s_norm;
'''

def fetch_from_neo4j(period_name, start, end, score_prop):
    try:
        from neo4j import GraphDatabase
    except Exception as e:
        raise RuntimeError("neo4j driver not installed. Run `pip install neo4j` in this environment.")
    params = {
        'sy': start['year'], 'sm': start['month'], 'sd': start['day'],
        'sh': start['hour'], 'smin': start['minute'], 'ss': start['second'],
        'ey': end['year'],   'em': end['month'],   'ed': end['day'],
        'eh': end['hour'],   'emin': end['minute'],'es': end['second'],
    }
    q = cypher_for_period(score_prop)
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS))
    with driver.session() as sess:
        rec = sess.run(q, **params).single()
    driver.close()
    s_anom = np.array([x for x in rec['s_anom'] if x is not None], dtype=float)
    s_norm = np.array([x for x in rec['s_norm'] if x is not None], dtype=float)
    return s_anom, s_norm

def prop_within_eps(x, eps):
    x = np.asarray(x)
    if x.size == 0:
        return np.nan
    return float(np.mean(np.abs(x) < eps))

def wasserstein_1(a, b):
    """Wasserstein-1 distance using numpy only (no SciPy dependency)."""
    a = np.sort(np.asarray(a, dtype=float))
    b = np.sort(np.asarray(b, dtype=float))
    if a.size == 0 or b.size == 0:
        return np.nan
    # Empirical CDF approach on the pooled grid
    grid = np.sort(np.concatenate([a, b]))
    # CDFs via searchsorted
    Fa = np.searchsorted(a, grid, side='right') / a.size
    Fb = np.searchsorted(b, grid, side='right') / b.size
    # Integral of |Fa-Fb| over grid using trapezoidal rule
    diffs = np.abs(Fa - Fb)
    return float(np.trapz(diffs, grid))

def cliffs_delta(a, b):
    """Cliff's delta (δ) computed via Mann–Whitney U equivalence (numpy-only).
    δ = 2U/(mn) − 1. We compute U from ranks.
    """
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    if a.size == 0 or b.size == 0:
        return np.nan
    x = np.concatenate([a, b])
    order = np.argsort(x, kind='mergesort')
    ranks = np.empty_like(order, dtype=float)
    ranks[order] = np.arange(1, x.size + 1)
    # tie correction: average ranks for ties
    # find runs of equal values in sorted x
    xs = x[order]
    i = 0
    while i < xs.size:
        j = i
        while j + 1 < xs.size and xs[j+1] == xs[i]:
            j += 1
        if j > i:
            avg = (i + 1 + j + 1) / 2.0
            ranks[i:j+1] = avg
        i = j + 1
    ra = ranks[:a.size]
    rb = ranks[a.size:]
    # Mann–Whitney U for a
    m, n = float(a.size), float(b.size)
    Ua = np.sum(ra) - m*(m+1)/2.0
    delta = 2*Ua/(m*n) - 1
    return float(delta)

def overlap_coefficient(a, b, bins=100):
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    if a.size == 0 or b.size == 0:
        return np.nan
    hist_a, edges = np.histogram(a, bins=bins, density=True)
    hist_b, _     = np.histogram(b, bins=edges, density=True)
    ovl = np.sum(np.minimum(hist_a, hist_b) * np.diff(edges))
    return float(ovl)

def sci_notation(x, sig=6):
    if x is None or (isinstance(x, float) and (math.isnan(x) or math.isinf(x))):
        return '—'
    if x == 0:
        return '0'
    sign = '-' if x < 0 else ''
    ax = abs(x)
    exp10 = int(math.floor(math.log10(ax)))
    mant = round(ax / (10 ** exp10), sig)
    return f"{sign}{mant} × 10^{exp10}"


### 2.1) Optional: paste arrays if not using Neo4j
If `DATA_SOURCE = 'manual'`, paste your arrays below as Python lists.

In [3]:
# Example placeholders (replace with your arrays if DATA_SOURCE == 'manual')
manual_data = {
    'April 1st 2025': {
        'anom': [],  # e.g., [-9.2e-9, -1.1e-8, ...]
        'norm': [],  # e.g., [1.9978e-6, 8.3e-7, ...]
    },
    'April 2025': {
        'anom': [],
        'norm': [],
    },
}


## 3) Compute metrics
For each period, we compute:
- Anomaly rate (share of negatives if `NEGATIVE_IS_ANOMALY=True`)
- Min/Max anomaly score (negatives) and Min/Max normal score (positives), formatted as scientific notation
- Wasserstein‑1 distance on raw scores and on `|score|`
- Cliff’s delta on `|score|`
- Overlap Coefficient on `|score|`
- Proportion within ε of 0 for anomalies and normals

In [4]:
results = []
for period_name, bounds in PERIODS.items():
    if DATA_SOURCE == 'neo4j':
        s_anom, s_norm = fetch_from_neo4j(period_name, bounds['start'], bounds['end'], SCORE_PROPERTY)
    else:
        s_anom = np.array(manual_data.get(period_name, {}).get('anom', []), dtype=float)
        s_norm = np.array(manual_data.get(period_name, {}).get('norm', []), dtype=float)

    # Basic counts and rates
    total = int(s_anom.size + s_norm.size)
    anomalies = int(s_anom.size) if NEGATIVE_IS_ANOMALY else int(s_norm.size)
    anomaly_rate = (100.0 * anomalies / total) if total > 0 else float('nan')

    # Extrema
    an_min = float(np.min(s_anom)) if s_anom.size else None
    an_max = float(np.max(s_anom)) if s_anom.size else None
    no_min = float(np.min(s_norm)) if s_norm.size else None
    no_max = float(np.max(s_norm)) if s_norm.size else None

    # Distances / effect sizes
    w_raw = wasserstein_1(s_anom, s_norm)
    w_abs = wasserstein_1(np.abs(s_anom), np.abs(s_norm))
    delta = cliffs_delta(np.abs(s_anom), np.abs(s_norm))
    ovl = overlap_coefficient(np.abs(s_anom), np.abs(s_norm), bins=100)

    # Proportions near zero
    within = {eps: (prop_within_eps(s_anom, eps), prop_within_eps(s_norm, eps)) for eps in EPSILONS}

    results.append({
        'Period': period_name,
        'N_total': total,
        'Anomaly rate (%)': round(anomaly_rate, 2) if not math.isnan(anomaly_rate) else float('nan'),
        'Anomaly Min Score': sci_notation(an_min),
        'Anomaly Max Score': sci_notation(an_max),
        'Normal Min Score': sci_notation(no_min),
        'Normal Max Score': sci_notation(no_max),
        'W1 (raw)': w_raw,
        'W1 (|score|)': w_abs,
        "Cliff's δ (|score|)": delta,
        'OVL (|score|)': ovl,
        'Within-ε (anom,norm)': within,
    })

results


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "\nWITH datetime({year:$sy,month:$sm,day:$sd,hour:$sh,minute:$smin,second:$ss, timezone:'America/Sao_Paulo'}) AS startBR,\n     datetime({year:$ey,month:$em,day:$ed,hour:$eh,minute:$emin,second:$es, timezone:'America/Sao_Paulo'}) AS endBR\nMATCH (tx:Transaction)\nWHERE tx.scoring IS NOT NULL\n  AND datetime({epochSeconds: tx.timestamp, timezone:'America/Sao_Paulo'}) >= startBR\n  AND datetime({epochSeconds: tx.timestamp, timezone:'America/Sao_Paulo'}) <  endBR\nRETURN\n  collect(CASE WHEN tx.scoring < 0 THEN tx.scoring END) AS s_anom,\n  collect(CASE WHEN tx.scoring > 0 THEN tx.scoring END) AS s_norm;\n"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

[{'Period': 'April 1st 2025',
  'N_total': 62758,
  'Anomaly rate (%)': 7.29,
  'Anomaly Min Score': '-2.01844 × 10^-1',
  'Anomaly Max Score': '-7.974794 × 10^-5',
  'Normal Min Score': '1.530396 × 10^-4',
  'Normal Max Score': '2.863329 × 10^-1',
  'W1 (raw)': 0.28804962771704434,
  'W1 (|score|)': 0.18393994210316017,
  "Cliff's δ (|score|)": -0.9901937355724609,
  'OVL (|score|)': 0.31196396400677384,
  'Within-ε (anom,norm)': {1e-06: (0.0, 0.0),
   1e-05: (0.0, 0.0),
   0.0001: (0.00043706293706293706, 0.0)}},
 {'Period': 'April 2025',
  'N_total': 3675439,
  'Anomaly rate (%)': 5.25,
  'Anomaly Min Score': '-2.346918 × 10^-1',
  'Anomaly Max Score': '-7.53023 × 10^-7',
  'Normal Min Score': '2.851699 × 10^-6',
  'Normal Max Score': '2.864268 × 10^-1',
  'W1 (raw)': 0.27892639862846197,
  'W1 (|score|)': 0.18893884652643342,
  "Cliff's δ (|score|)": -0.9954564090399018,
  'OVL (|score|)': 0.22037343546813237,
  'Within-ε (anom,norm)': {1e-06: (1.037059314607499e-05, 0.0),
   1e-05

## 4) Display a thesis‑ready summary table
Shows the core fields you need, with scores in scientific notation.

In [5]:
import pandas as pd
display_cols = [
    'Period', 'Anomaly rate (%)', 'Anomaly Min Score', 'Anomaly Max Score',
    'Normal Min Score', 'Normal Max Score', 'W1 (|score|)', "Cliff's δ (|score|)", 'OVL (|score|)'
]
df = pd.DataFrame(results)[display_cols]
df


,Period,Anomaly rate (%),Anomaly Min Score,Anomaly Max Score,Normal Min Score,Normal Max Score,W1 (|score|),Cliff's δ (|score|),OVL (|score|)
0,April 1st 2025,7.29,-2.01844 × 10^-1,-7.974794 × 10^-5,1.530396 × 10^-4,2.863329 × 10^-1,0.183940,-0.990194,0.311964
1,April 2025,5.25,-2.346918 × 10^-1,-7.53023 × 10^-7,2.851699 × 10^-6,2.864268 × 10^-1,0.188939,-0.995456,0.220373


## 5) Thesis‑style prose (auto‑filled from the metrics)
Below, a short paragraph is generated for each period. You can paste it directly into the thesis.

In [6]:
def fmt_pct(x):
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return '—'
    # Brazilian decimal comma
    return f"{x:.2f}".replace('.', ',') + '%'

paras = []
for r in results:
    period = r['Period']
    ar = fmt_pct(r['Anomaly rate (%)'])
    an_min, an_max = r['Anomaly Min Score'], r['Anomaly Max Score']
    no_min, no_max = r['Normal Min Score'], r['Normal Max Score']
    wabs = r['W1 (|score|)']
    delta = r["Cliff's δ (|score|)"]
    ovl = r['OVL (|score|)']
    para = (
        f"In {period}, the anomaly rate was {ar}. The score ranges were "
        f"anomalies: [{an_min}, {an_max}] and normals: [{no_min}, {no_max}]. "
        f"Separation beyond the sign threshold remained limited, with a 1‑Wasserstein distance on |score| of {wabs:.3e}, "
        f"Cliff’s delta of {delta:.3f}, and an overlap coefficient of {ovl:.3f}."
    )
    paras.append(para)
for p in paras:
    print('- ' + p)


- In April 1st 2025, the anomaly rate was 7,29%. The score ranges were anomalies: [-2.01844 × 10^-1, -7.974794 × 10^-5] and normals: [1.530396 × 10^-4, 2.863329 × 10^-1]. Separation beyond the sign threshold remained limited, with a 1‑Wasserstein distance on |score| of 1.839e-01, Cliff’s delta of -0.990, and an overlap coefficient of 0.312.
- In April 2025, the anomaly rate was 5,25%. The score ranges were anomalies: [-2.346918 × 10^-1, -7.53023 × 10^-7] and normals: [2.851699 × 10^-6, 2.864268 × 10^-1]. Separation beyond the sign threshold remained limited, with a 1‑Wasserstein distance on |score| of 1.889e-01, Cliff’s delta of -0.995, and an overlap coefficient of 0.220.


## 6) (Optional) ε‑neighborhood report around zero
Reports the share of points from each class that lie within ε of zero, for interpretability.

In [7]:
rows = []
for r in results:
    period = r['Period']
    for eps, (pa, pn) in r['Within-ε (anom,norm)'].items():
        rows.append({
            'Period': period,
            'ε': eps,
            'P(|score|<ε | anomaly)': pa,
            'P(|score|<ε | normal)': pn,
        })
eps_df = pd.DataFrame(rows)
eps_df


,Period,ε,P(|score|<ε | anomaly),P(|score|<ε | normal)
0,April 1st 2025,0.000001,0.000000,0.000000
1,April 1st 2025,0.000010,0.000000,0.000000
2,April 1st 2025,0.000100,0.000437,0.000000
3,April 2025,0.000001,0.000010,0.000000
4,April 2025,0.000010,0.000047,0.000004
5,April 2025,0.000100,0.001898,0.000052


## 7) Appendix — Cypher used (America/Sao_Paulo)
Copy/paste into Neo4j Browser if you want to validate the arrays independently.

In [8]:
print(cypher_for_period(SCORE_PROPERTY))


WITH datetime({year:$sy,month:$sm,day:$sd,hour:$sh,minute:$smin,second:$ss, timezone:'America/Sao_Paulo'}) AS startBR,
     datetime({year:$ey,month:$em,day:$ed,hour:$eh,minute:$emin,second:$es, timezone:'America/Sao_Paulo'}) AS endBR
MATCH (tx:Transaction)
WHERE tx.scoring IS NOT NULL
  AND datetime({epochSeconds: tx.timestamp, timezone:'America/Sao_Paulo'}) >= startBR
  AND datetime({epochSeconds: tx.timestamp, timezone:'America/Sao_Paulo'}) <  endBR
RETURN
  collect(CASE WHEN tx.scoring < 0 THEN tx.scoring END) AS s_anom,
  collect(CASE WHEN tx.scoring > 0 THEN tx.scoring END) AS s_norm;



## References (for footnotes)
- Liu, F. T., Ting, K. M., & Zhou, Z.-H. (2008). Isolation Forest. *ICDM 2008*.
- scikit‑learn developers. *IsolationForest — User Guide & API*. (document the zero‑threshold offset behavior; cite the docs/source).

> Note: A peer‑reviewed paper explicitly documenting scikit‑learn’s zero‑centering (`decision_function = score_samples − offset_`) is not available; cite the library documentation and source to support implementation details.
